# 2. 파이썬으로 RNN 구현하기

![RNN_구현하기](./data/RNN_구현하기.JPG)

In [1]:
import numpy as np

timesteps = 10 # 시점의 수. NLP에서는 보통 문장의 길이가 된다.
input_size = 4 # 입력의 차원. NLP에서는 보통 단어 벡터의 차원이 된다.
hidden_size = 8 # 은닉 상태의 크기. 메모리 셀의 용량이다.

# 입력에 해당되는 2D 텐서
inputs = np.random.random((timesteps, input_size)) 

# 초기 은닉 상태는 0(벡터)로 초기화
# 은닉 상태의 크기 hidden_size로 은닉 상태를 만듬.
hidden_state_t = np.zeros((hidden_size,)) 

# 8의 크기를 가지는 은닉 상태. 현재는 초기 은닉 상태로 모든 차원이 0의 값을 가짐.
print(hidden_state_t) 

[0. 0. 0. 0. 0. 0. 0. 0.]


In [2]:
# (8, 4)크기의 2D 텐서 생성. 입력에 대한 가중치.
Wx = np.random.random((hidden_size, input_size))

# (8, 8)크기의 2D 텐서 생성. 은닉 상태에 대한 가중치.
Wh = np.random.random((hidden_size, hidden_size))

# (8,)크기의 1D 텐서 생성. 이 값은 편향(bias).
b = np.random.random((hidden_size,)) 

# 가중치와 편향의 크기
print(np.shape(Wx))
print(np.shape(Wh))
print(np.shape(b))

(8, 4)
(8, 8)
(8,)


> Wx : (은닉 상태의 크기 x 입력의 차원)  
> Wh : (은닉 상태의 크기 X 은닉 상태의 크기)  
>  b : (은닉 상태의 크기)

In [3]:
total_hidden_states = []

# 메모리 셀 동작
# 각 시점에 따라서 입력값이 입력됨.
for input_t in inputs: 
    # Wx * Xt + Wh * Ht-1 + b(bias)
    output_t = np.tanh(np.dot(Wx,input_t) + np.dot(Wh,hidden_state_t) + b) 
    # 각 시점의 은닉 상태의 값을 계속해서 축적
    total_hidden_states.append(list(output_t)) 
    # 각 시점 t별 메모리 셀의 출력의 크기는 (timestep, output_dim)
    print(np.shape(total_hidden_states)) 
    hidden_state_t = output_t

# 출력 시 값을 깔끔하게 해준다.
total_hidden_states = np.stack(total_hidden_states, axis = 0) 


# (timesteps, output_dim)의 크기. 
# 이 경우 (10, 8)의 크기를 가지는 메모리 셀의 2D 텐서를 출력.
print(total_hidden_states)

(1, 8)
(2, 8)
(3, 8)
(4, 8)
(5, 8)
(6, 8)
(7, 8)
(8, 8)
(9, 8)
(10, 8)
[[0.60132174 0.7984977  0.93468321 0.88631126 0.82048706 0.7370612
  0.82975134 0.621055  ]
 [0.9997663  0.9995137  0.9999665  0.99940271 0.99995254 0.99589024
  0.99974435 0.99926305]
 [0.99999687 0.9999937  0.99999923 0.99998845 0.99999686 0.99984977
  0.99999588 0.99995552]
 [0.99999531 0.99999673 0.99999887 0.99999049 0.99999715 0.99987632
  0.99999835 0.99994846]
 [0.99998604 0.9999815  0.9999983  0.99995221 0.99999691 0.99969271
  0.99999491 0.9999171 ]
 [0.99999482 0.9999929  0.99999897 0.9999877  0.99999702 0.99983997
  0.99999495 0.99993877]
 [0.99999172 0.99998132 0.99999665 0.99996146 0.99999528 0.99943952
  0.99998912 0.9999206 ]
 [0.99998858 0.99997221 0.99999773 0.99995205 0.9999961  0.99952963
  0.99998628 0.99990996]
 [0.99999574 0.99999016 0.99999803 0.99998251 0.99999569 0.99967275
  0.99999174 0.99994075]
 [0.99999607 0.99999485 0.9999984  0.9999872  0.99999629 0.99978696
  0.99999674 0.99994905]]

# 3. 파이토치의 nn.RNN()

In [4]:
import torch
import torch.nn as nn

In [5]:
input_size = 5 # 입력의 크기
hidden_size = 8 # 은닉 상태의 크기

# (batch_size, time_steps, input_size)
inputs = torch.Tensor(1, 10, 5)

# batch_first=True : 입력 텐서의 첫번째 차원이 배치 크기임
cell = nn.RNN(input_size, hidden_size, batch_first=True)

# 입력 텐서를 RNN 셀에 입력하여 출력을 확인
outputs, _status = cell(inputs)
print(outputs.shape) # 모든 time-step의 hidden_state
print(_status.shape) # 최종 time-step의 hidden_state

torch.Size([1, 10, 8])
torch.Size([1, 1, 8])


> 입력 텐서 : (배치 크기 x 시점의 수 x 매 시점마다 들어가는 입력)

# 4. 깊은 순환 신경망(Deep Recurrent Neural Network)

In [6]:
# (batch_size, time_steps, input_size)
inputs = torch.Tensor(1, 10, 5)

# num_layers에 값을 전달하여 층을 쌓음
cell = nn.RNN(input_size = 5, hidden_size = 8, num_layers = 2, batch_first=True)

# 입력 텐서를 RNN 셀에 입력하여 출력을 확인
outputs, _status = cell(inputs)
print(outputs.shape) # 모든 time-step의 hidden_state
print(_status.shape) # (층의 개수, 배치 크기, 은닉 상태의 크기)


torch.Size([1, 10, 8])
torch.Size([2, 1, 8])


# 5. 양방향 순환 신경망(Bidirectional Recurrent Neural Network)

In [7]:
# (batch_size, time_steps, input_size)
inputs = torch.Tensor(1, 10, 5)

# bidirectional에 값을 True : 양방향 순환 신경망
cell = nn.RNN(input_size = 5, hidden_size = 8, num_layers = 2, batch_first=True, bidirectional = True)

# 입력 텐서를 RNN 셀에 입력하여 출력을 확인
outputs, _status = cell(inputs)
print(outputs.shape) # (배치 크기, 시퀀스 길이, 은닉 상태의 크기 x 2)
print(_status.shape) # (층의 개수 x 2, 배치 크기, 은닉 상태의 크기)

torch.Size([1, 10, 16])
torch.Size([4, 1, 8])


# 6. 장단기 메모리(Long Short-Term Memory, LSTM)

In [8]:
input_dim = 5
hidden_size = 8

- 기존의 RNN 셀

In [9]:
nn.RNN(input_dim, hidden_size, batch_first=True)

RNN(5, 8, batch_first=True)

- LSTM 셀

In [10]:
nn.LSTM(input_dim, hidden_size, batch_first=True) 

LSTM(5, 8, batch_first=True)